In [1]:
# TensorFlow and tf.keras
import tensorflow as tf

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import regularizers
from PIL import Image

In [2]:
# Chemin vers le répertoire contenant les images d'entraînement
train_dir = r"C:\\Users\\marti\\OneDrive\\Martin OneDrive\\ISEP\\A2\\Semestre 2\\Ai and optimisation\\Projet\train\\Test 28-04-2023\\Entrainement\\"
test_dir = r"C:\\Users\\marti\\OneDrive\\Martin OneDrive\\ISEP\\A2\\Semestre 2\\Ai and optimisation\\Projet\train\\Test 28-04-2023\\Test\\"

# Paramètres du modèle
batch_size = 32
img_height = 256
img_width = 256
epochs = 50

print("Prétraitement des données...")
# Créer un générateur de données pour l'entraînement
train_datagen = ImageDataGenerator(    
    rescale=1./255,
    rotation_range=90,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')


# Construction du modèle
print("Construction du modèle...")
model = Sequential([
    Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(img_height, img_width ,3)),
    MaxPooling2D(),
    Conv2D(64, (3, 3), padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(128, (3, 3), padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(256, (3, 3), padding='same', activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(train_generator.class_indices), activation='softmax')
])
# Compiler le modèle
print("Compilation du modèle...")
#optimizertest = Adam(learning_rate=0.01)  # Optimiseur avec un taux d'apprentissage plus faible
model.compile(optimizer='adam',
              loss='categorical_crossentropy',  # Pour la classification multiclasse
              metrics=['accuracy'])

model.summary()

# Entraîner le modèle
print("Entraînement du modèle...")
model.fit(
    train_generator,
    epochs=epochs
)

# Évaluation du modèle
print("Évaluation du modèle...")
loss, accuracy = model.evaluate(train_generator)
print("Précision sur les données d'entraînement:", accuracy)

# Sauvegarde du modèle
print("Sauvegarde du modèle...")
model.save('insect_classifier_model')
print("Modèle sauvegardé sous le nom 'insect_classifier_model'")

Prétraitement des données...
Found 185 images belonging to 6 classes.
Construction du modèle...


Compilation du modèle...

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 128, 128, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 64, 64, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 64, 64, 128)       73856    

6/6 [==============================] - 11s 2s/step - loss: 0.4832 - accuracy: 0.8378
Epoch 49/50
6/6 [==============================] - 11s 2s/step - loss: 0.5622 - accuracy: 0.7838
Epoch 50/50
6/6 [==============================] - 10s 2s/step - loss: 0.6570 - accuracy: 0.8000
Évaluation du modèle...
6/6 [==============================] - 8s 1s/step - loss: 0.5285 - accuracy: 0.8000
Précision sur les données d'entraînement: 0.800000011920929
Sauvegarde du modèle...
INFO:tensorflow:Assets written to: insect_classifier_model\assets


INFO:tensorflow:Assets written to: insect_classifier_model\assets


Modèle sauvegardé sous le nom 'insect_classifier_model'


In [3]:
from tensorflow.keras.models import load_model
import os
import numpy as np
from tensorflow.keras.preprocessing import image

# Charger le modèle entraîné
model = load_model('insect_classifier_model')

# Chemin vers le dossier de test
test_folder = r"C:\\Users\\marti\\OneDrive\\Martin OneDrive\\ISEP\\A2\\Semestre 2\\Ai and optimisation\\Projet\train\\Test 28-04-2023\\Test\\"

# Prétraitement des images de test
test_images = []
for filename in os.listdir(test_folder):
    img = image.load_img(os.path.join(test_folder, filename), target_size=(img_height, img_width))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Ajouter une dimension pour le lot
    test_images.append(img_array)

# Concaténer toutes les images en un seul tableau numpy
test_images = np.vstack(test_images)

# Prédictions
predictions = model.predict(test_images)

# Afficher les prédictions
for i, filename in enumerate(os.listdir(test_folder)):
    print("Image:", filename)
    predicted_class = np.argmax(predictions[i])
    print("Classe prédite:", predicted_class)
    print()
    
import openpyxl

# Créer un nouveau classeur Excel
wb = openpyxl.Workbook()

# Sélectionner la feuille active
ws = wb.active

# Entête
ws['A1'] = 'Nom de l\'image'
ws['B1'] = 'Espèce prédite'
ws['C1'] = 'Pourcentage de confiance'

# Écrire les prédictions dans le fichier Excel
for i, filename in enumerate(os.listdir(test_folder)):
    ws[f'A{i+2}'] = filename
    predicted_class = np.argmax(predictions[i])
    ws[f'B{i+2}'] = predicted_class
    confidence = predictions[i][predicted_class] * 100
    ws[f'C{i+2}'] = f"{confidence:.2f}%"

# Sauvegarder le classeur Excel
wb.save('prediction2.xlsx')

3/3 [==============================] - 1s 135ms/step
Image: IMG_ZX_175.JPG
Classe prédite: 2

Image: IMG_ZX_176.JPG
Classe prédite: 2

Image: IMG_ZX_177.JPG
Classe prédite: 1

Image: IMG_ZX_178.JPG
Classe prédite: 1

Image: IMG_ZX_179.JPG
Classe prédite: 2

Image: IMG_ZX_183.JPG
Classe prédite: 2

Image: IMG_ZX_184.JPG
Classe prédite: 2

Image: IMG_ZX_185.JPG
Classe prédite: 2

Image: IMG_ZX_186.JPG
Classe prédite: 2

Image: IMG_ZX_187.JPG
Classe prédite: 2

Image: IMG_ZX_188.JPG
Classe prédite: 2

Image: IMG_ZX_190.JPG
Classe prédite: 2

Image: IMG_ZX_191.JPG
Classe prédite: 2

Image: IMG_ZX_193.JPG
Classe prédite: 2

Image: IMG_ZX_194.JPG
Classe prédite: 2

Image: IMG_ZX_195.JPG
Classe prédite: 2

Image: IMG_ZX_196.JPG
Classe prédite: 1

Image: IMG_ZX_197.JPG
Classe prédite: 2

Image: IMG_ZX_198.JPG
Classe prédite: 1

Image: IMG_ZX_199.JPG
Classe prédite: 5

Image: IMG_ZX_200.JPG
Classe prédite: 2

Image: IMG_ZX_201.JPG
Classe prédite: 5

Image: IMG_ZX_202.JPG
Classe prédite: 2

Imag